In [12]:
# from softcomputing_packages_bu import *
from apyori import apriori
import pandas as pd
import numpy as np
import copy
import networkx as nx
import matplotlib.pyplot as plt

In [13]:
# Load dataset
hc_csv_raw = pd.read_csv('./datasets/2.0-discretized-v2-2-2-with-label-resampled-and-DLP-encoded.csv')
hc_csv_raw.head()
len(hc_csv_raw)

41673

In [14]:
# Sampling (for Development)
n_sample = 30000
hc_csv = copy.copy(hc_csv_raw)
hc_csv.head()

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
0,Very High,Low,Low,Very High,High,Medium,High,Very High,True,march,spring,saturday,no_significant_change,early_attention,cam_1,False,True,False
1,High,Low,High,High,Low,Low,High,High,False,january,winter,thursday,no_significant_change,other,cam_1,False,True,False
2,Very High,High,High,Very High,Medium,Medium,Very High,Very High,False,january,winter,friday,no_significant_change,early_attention,cam_1,False,True,False
3,Low,Low,Low,Low,Medium,Low,Low,Medium,False,february,winter,friday,no_significant_change,late_attention,cam_2,False,True,False
4,Very High,Low,Very High,Very High,Low,Low,Low,Very High,True,december,winter,saturday,no_significant_change,early_attention,cam_1,False,True,False


In [15]:
# Preprocess dataset
hc_csv_tmp = copy.copy(hc_csv)
for c in range(8):
    print(c)
    print("[%d] Column Preprocessing: %s" %(c, hc_csv_tmp.columns[c]))
    for r in range(hc_csv_tmp.shape[0]):
        if(hc_csv_tmp.iloc[r,c]=='Very High'):
            hc_csv_tmp.iloc[r,c] = hc_csv_tmp.columns[c]+' 75-100%'
        elif(hc_csv_tmp.iloc[r,c]=='High'):
            hc_csv_tmp.iloc[r,c] = hc_csv_tmp.columns[c]+' 50-75%'
        elif(hc_csv_tmp.iloc[r,c]=='Medium'):
            hc_csv_tmp.iloc[r,c] = hc_csv_tmp.columns[c]+' 25-50%'
        elif(hc_csv_tmp.iloc[r,c]=='Low'):
            # hc_csv_tmp.iloc[r,c] = '-1'
            hc_csv_tmp.iloc[r,c] = hc_csv_tmp.columns[c]+' 0-25%'
        else:
            hc_csv_tmp.iloc[r,c] = '-1'
print("Done.")
hc_csv = copy.copy(hc_csv_tmp)

0
[0] Column Preprocessing: global_active_power
1
[1] Column Preprocessing: global_reactive_power
2
[2] Column Preprocessing: voltage
3
[3] Column Preprocessing: global_intensity
4
[4] Column Preprocessing: kitchen
5
[5] Column Preprocessing: laundry
6
[6] Column Preprocessing: climate_control
7
[7] Column Preprocessing: other
Done.


In [70]:
hc_csv_tmp.iloc[:,1].name

'global_reactive_power'

In [74]:
hc_csv_tmp = copy.copy(hc_csv)

for c in range(len(hc_csv_tmp.columns)-3):
    if (c > 7):
        print(f'[{c}] Processing col: {hc_csv_tmp.iloc[:,c].name}...')
        for i in range(len(hc_csv_tmp)):
            # print(c)
            hc_csv_tmp.iloc[i,c] = hc_csv_tmp.iloc[:,c].name + ' ' + hc_csv_tmp.iloc[i,c]

print('done')  
hc_csv_tmp

[8] Processing col: weekend...
[9] Processing col: month_name...
[10] Processing col: season_name...
[11] Processing col: day_name...
[12] Processing col: peak_label_pred...
[13] Processing col: attention_type_label...
[14] Processing col: activation_type_label...
done


,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
0,global_active_power 75-100%,global_reactive_power 0-25%,voltage 0-25%,global_intensity 75-100%,kitchen 50-75%,laundry 25-50%,climate_control 50-75%,other 75-100%,weekend weekend True,month_name march,season_name spring,day_name saturday,peak_label_pred no_significant_change,attention_type_label early_attention,activation_type_label cam_1,False,True,False
1,global_active_power 50-75%,global_reactive_power 0-25%,voltage 50-75%,global_intensity 50-75%,kitchen 0-25%,laundry 0-25%,climate_control 50-75%,other 50-75%,weekend weekend False,month_name january,season_name winter,day_name thursday,peak_label_pred no_significant_change,attention_type_label other,activation_type_label cam_1,False,True,False
2,global_active_power 75-100%,global_reactive_power 50-75%,voltage 50-75%,global_intensity 75-100%,kitchen 25-50%,laundry 25-50%,climate_control 75-100%,other 75-100%,weekend weekend False,month_name january,season_name winter,day_name friday,peak_label_pred no_significant_change,attention_type_label early_attention,activation_type_label cam_1,False,True,False
3,global_active_power 0-25%,global_reactive_power 0-25%,voltage 0-25%,global_intensity 0-25%,kitchen 25-50%,laundry 0-25%,climate_control 0-25%,other 25-50%,weekend weekend False,month_name february,season_name winter,day_name friday,peak_label_pred no_significant_change,attention_type_label late_attention,activation_type_label cam_2,False,True,False
4,global_active_power 75-100%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 75-100%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 75-100%,weekend weekend True,month_name december,season_name winter,day_name saturday,peak_label_pred no_significant_change,attention_type_label early_attention,activation_type_label cam_1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41668,global_active_power 0-25%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 0-25%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 25-50%,weekend weekend True,month_name january,season_name winter,day_name saturday,peak_label_pred lower_than_usual,attention_type_label late_attention,activation_type_label cam_2,False,False,True
41669,global_active_power 25-50%,global_reactive_power 25-50%,voltage 75-100%,global_intensity 25-50%,kitchen 50-75%,laundry 25-50%,climate_control 25-50%,other 50-75%,weekend weekend False,month_name february,season_name winter,day_name tuesday,peak_label_pred lower_than_usual,attention_type_label other,activation_type_label cam_2,False,False,True
41670,global_active_power 25-50%,global_reactive_power 75-100%,voltage 50-75%,global_intensity 25-50%,kitchen 0-25%,laundry 50-75%,climate_control 0-25%,other 50-75%,weekend weekend False,month_name april,season_name spring,day_name tuesday,peak_label_pred lower_than_usual,attention_type_label other,activation_type_label cam_2,False,False,True
41671,global_active_power 0-25%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 0-25%,kitchen 50-75%,laundry 50-75%,climate_control 25-50%,other 0-25%,weekend weekend True,month_name january,season_name winter,day_name saturday,peak_label_pred lower_than_usual,attention_type_label late_attention,activation_type_label cam_2,False,False,True


In [75]:
hc_csv_tmp.to_csv(path_or_buf='./datasets/2.0-discretized-v2-2-2-1-preprocessed.csv', index=False)

In [76]:
pd.read_csv('./datasets/2.0-discretized-v2-2-2-1-preprocessed.csv')

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
0,global_active_power 75-100%,global_reactive_power 0-25%,voltage 0-25%,global_intensity 75-100%,kitchen 50-75%,laundry 25-50%,climate_control 50-75%,other 75-100%,weekend weekend True,month_name march,season_name spring,day_name saturday,peak_label_pred no_significant_change,attention_type_label early_attention,activation_type_label cam_1,False,True,False
1,global_active_power 50-75%,global_reactive_power 0-25%,voltage 50-75%,global_intensity 50-75%,kitchen 0-25%,laundry 0-25%,climate_control 50-75%,other 50-75%,weekend weekend False,month_name january,season_name winter,day_name thursday,peak_label_pred no_significant_change,attention_type_label other,activation_type_label cam_1,False,True,False
2,global_active_power 75-100%,global_reactive_power 50-75%,voltage 50-75%,global_intensity 75-100%,kitchen 25-50%,laundry 25-50%,climate_control 75-100%,other 75-100%,weekend weekend False,month_name january,season_name winter,day_name friday,peak_label_pred no_significant_change,attention_type_label early_attention,activation_type_label cam_1,False,True,False
3,global_active_power 0-25%,global_reactive_power 0-25%,voltage 0-25%,global_intensity 0-25%,kitchen 25-50%,laundry 0-25%,climate_control 0-25%,other 25-50%,weekend weekend False,month_name february,season_name winter,day_name friday,peak_label_pred no_significant_change,attention_type_label late_attention,activation_type_label cam_2,False,True,False
4,global_active_power 75-100%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 75-100%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 75-100%,weekend weekend True,month_name december,season_name winter,day_name saturday,peak_label_pred no_significant_change,attention_type_label early_attention,activation_type_label cam_1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41668,global_active_power 0-25%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 0-25%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 25-50%,weekend weekend True,month_name january,season_name winter,day_name saturday,peak_label_pred lower_than_usual,attention_type_label late_attention,activation_type_label cam_2,False,False,True
41669,global_active_power 25-50%,global_reactive_power 25-50%,voltage 75-100%,global_intensity 25-50%,kitchen 50-75%,laundry 25-50%,climate_control 25-50%,other 50-75%,weekend weekend False,month_name february,season_name winter,day_name tuesday,peak_label_pred lower_than_usual,attention_type_label other,activation_type_label cam_2,False,False,True
41670,global_active_power 25-50%,global_reactive_power 75-100%,voltage 50-75%,global_intensity 25-50%,kitchen 0-25%,laundry 50-75%,climate_control 0-25%,other 50-75%,weekend weekend False,month_name april,season_name spring,day_name tuesday,peak_label_pred lower_than_usual,attention_type_label other,activation_type_label cam_2,False,False,True
41671,global_active_power 0-25%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 0-25%,kitchen 50-75%,laundry 50-75%,climate_control 25-50%,other 0-25%,weekend weekend True,month_name january,season_name winter,day_name saturday,peak_label_pred lower_than_usual,attention_type_label late_attention,activation_type_label cam_2,False,False,True
